# OliveYoung_올리브영

제한된 수의 `카운터`, 제한된 수의 `종업원`과 랜덤하게 도착하는 `고객`으로 이루어진 올리브영 시뮬레이션 환경을 생성합니다.

이 시뮬레이션에서 매장에는 여러 명의 종업원이 있으며, 종업원은 카운터와 상품 진열, 안내 업무를 랜덤하게 담당합니다. 종업원마다 정해진 업무가 없다는 뜻입니다. 고객은 쇼핑하는데 특정 시간이 걸리며, 결제를 위해 카운터 대기열에 들어갑니다.

시뮬레이션을 수정하여 고객 도착율, 대기열 및 계산대의 용량 등을 조정할 수 있습니다. 또한, 대기 시간과 구매량, 각 계산대별 거래량 등의 추가 통계 정보를 수집할 수 있습니다.

In [15]:
import simpy
import random

# 올리브영 시뮬레이션
class OliveYoungSimulation:
    def __init__(self, env, num_counters, num_guides, shopping_time_mean, shopping_time_std,service_time_mean, service_time_std):
        self.env = env
        self.counters = [simpy.Resource(env) for _ in range(num_counters)]
        self.guides = [simpy.Resource(env) for _ in range(num_guides)]
        self.shopping_time_mean = shopping_time_mean
        self.shopping_time_std = shopping_time_std
        self.service_time_mean = service_time_mean
        self.service_time_std = service_time_std

    # 주문 처리
    def process_order(self, customer):
        # 일하는 카운터 선택
        counter_idx = random.randint(0, len(self.counters)-1)
        counter = self.counters[counter_idx]
        
        # 안내 종업원 선택
        guide = random.choice(self.guides)
        with guide.request() as req:
            yield req
            shopping_time = random.gauss(mu=self.shopping_time_mean, sigma=self.shopping_time_std)
            yield self.env.timeout(shopping_time)
            print("======(똑같은 말투와 힘빠진 표정으로) 찾으시는 상품이 있으시면 말씀해주세요======")

        # 카운터 큐에서 대기 중인 고객 수
        queue_len = len(counter.queue)
        print(f'{self.env.now:.2f}: {customer}님이 {counter_idx}번 카운터에 대기합니다.(대기열 {queue_len}명)')

        with counter.request() as req:
            yield req
            # 결제 종업원 선택
            guide = random.choice(self.guides)
            with guide.request() as req:
                yield req
                # 결제 시간
                print(f'{self.env.now:.2f}: {customer}님 CJ 포인트 적입하시나요?')
                service_time = random.gauss(mu=self.service_time_mean, sigma=self.service_time_std)
                yield self.env.timeout(service_time)
                print(f'{self.env.now:.2f}: (결제 {service_time:.1f}분 소요){customer}님 영수증은 앱으로 확인 가능하십니다. 감사합니다:)')


In [16]:
    # 고객 생성
    def generate_customers(self, interarrival_time_mean):
        customer_count = 0
        while True:
            # 고객 도착 시간
            interarrival_time = random.expovariate(1/interarrival_time_mean)
            yield self.env.timeout(interarrival_time)
            customer = f'{customer_count}번 고객'
            customer_count += 1
            print(f'{self.env.now:.2f}: ------{customer} 입장------')
            self.env.process(self.process_order(customer))

In [ ]:
# 시뮬레이션 파라미터 설정
num_counters = 2  # 카운터 수
num_guides = 5  # 안내 종업원 수
shopping_time_mean = 2.0
shopping_time_std = 0.5
service_time_mean = 2.0  # 결제 시간 평균
service_time_std = 0.5  # 결제 시간 표준편차
interarrival_time_mean = 1.0  # 고객 도착 평균 간격

# 올리브영 시뮬레이션 실행
print('########### 올리브영 시뮬레이션 시작! ##############')
env = simpy.Environment()
oliveyoung = OliveYoungSimulation(env, num_counters, num_guides, shopping_time_mean, shopping_time_std, service_time_mean, service_time_std)
env.process(oliveyoung.generate_customers(interarrival_time_mean))
env.run(until=50.0)  # 시뮬레이션 시간